# Agent Using smolagents

In the last section, we learned how we can create Agents from scratch using Python code, and we saw just how tedious that process can be. Fortunately, many Agent libraries simplify this work by handling much of the heavy lifting for you.

In this tutorial, you’ll create your very first Agent capable of performing actions such as image generation, web search, time zone checking and much more!

You will also publish your agent on a Hugging Face Space so you can share it with friends and colleagues.

Let’s get started!

## What is smolagents?

`smolagents` is a library that provides a framework for developing your agents with ease. It's designed for simplicity and abstracts away much of the complexity of building an Agent.

**In short, `smolagents` is a library that focuses on codeAgent, a kind of agent that performs “Actions” through code blocks, and then “Observes” results by executing the code.**

In [6]:
from dotenv import load_dotenv
import os
from smolagents import CodeAgent, DuckDuckGoSearchTool, FinalAnswerTool, InferenceClientModel, load_tool, tool
import datetime
import requests
import pytz
import yaml

load_dotenv()
token = os.getenv("HF_INF_TOKEN")